<a href="https://colab.research.google.com/github/shalinikumari50/Web-Scrapping/blob/master/WebScraperStackExchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv


In [ ]:
CategoryList = [
                "https://datascience.stackexchange.com/questions/tagged/machine-learning?tab=newest&page=1&pagesize=50", 
                
                ]


In [ ]:
import math
import re
import time


for category in CategoryList:
            rowList = []
            baseLinkIndex = category.find("/questions/")
            pageLinkIndex = category.find("1")  
            
            categoryHTML = requests.get(category,headers = {'User-agent': 'your bot 0.1'})
            if not categoryHTML.ok:
                    print("scrapping stopped of a category")
                    time.sleep(300)
                    continue
            
            parser = BeautifulSoup(categoryHTML.text, 'html.parser')
            pages = parser.find('div',class_="fs-body3 grid--cell fl1 mr12 sm:mr0 sm:mb12").getText()
            noOfPages = int("".join(map(str, re.findall('\d+', pages))))
            noOfPages = math.ceil(noOfPages/50)
            
            for i in range(1,noOfPages+1):
              print("page number ------------------------------------" + str(i))
              pageLink = category[:pageLinkIndex]+ str(i) + category[pageLinkIndex+1:]
             
              pageHTML = requests.get(pageLink,headers = {'User-agent': 'your bot 0.1'})
              if not pageHTML.ok:
                    print("scrapping stopped of a page")
                    time.sleep(200)
                    continue
              parser = BeautifulSoup(pageHTML.text, 'html.parser')
              questionLinks = parser.find_all('a',class_="question-hyperlink")
              for questionLink in questionLinks:
                  anchor = questionLink.get('href')
                  if not 'questions' in anchor or 'https://' in anchor:
                        continue
                  link=category[:baseLinkIndex]+questionLink.get('href')
                  
                  url = requests.get(link,headers = {'User-agent': 'your bot 0.1'})
                
                  if not url.ok:
                    print("scrapping stopped of a question")
                    time.sleep(30)
                    break
                  ques = BeautifulSoup(url.text,'html.parser')
                  post = ques.find('div', class_="post-text")
                  maths = post.find_all('span',class_="math-container")
                  codes = post.find_all('code')
                  for math in maths:
                    math.extract()
                  for code in codes:
                    code.extract() 
                  posts = post.getText()
                  cleaned_post = ''.join([x.replace("\n"," ") for x in posts])
                  tags = ques.find_all('a', class_="post-tag",rel = 'tag')
                  tagList = []
                  for t in tags:
                    tagList.append(t.getText())
                  
                  
                  row = {'link': link, 'post':cleaned_post,'Tags' : set(tagList)}
                  print(row)
                  rowList.append(row)
                  scraped_data = pd.DataFrame(rowList)
                  catIndex = category.find(".")
                  tagIndex = category.find("/tagged/")
                  tagEndIndex = category.find("?")
                  name = category[8:catIndex] + "_" +category[tagIndex+8:tagEndIndex] + ".csv"
                  scraped_data.to_csv(name)                
